In [1]:
import os
import sys
import re
from copy import deepcopy
from typing import List, Dict, Tuple
from collections import OrderedDict
from glob import glob
import json
import yaml

from tqdm.auto import tqdm
import cv2
from PIL import Image
import numpy as np

sys.path.append("/home/rime97410000/ZebraFish_Code/ZebraFish_AP_POS/modules") # add path to scan customized module
from fileop import create_new_dir
from gallery_utils import draw_x_on_image, draw_drop_info_on_image
from datasetop import sortFishNameForDataset
import plt_show

# print("="*100, "\n")

Load `make_crop_gallery.yaml`

In [2]:
with open("make_crop_gallery.yaml", mode="r") as f_reader:
    config = yaml.load(f_reader, Loader=yaml.SafeLoader)

column = config["column"]

line_color = config["draw"]["line"]["color"]
line_width = config["draw"]["line"]["width"]

text_selected_color = config["draw"]["text"]["color"]["selected"]
text_drop_color     = config["draw"]["text"]["color"]["drop"]
text_shadow_color   = config["draw"]["text"]["color"]["shadow"]
text_font_style = config["draw"]["text"]["font_style"]
text_font_size  = config["draw"]["text"]["font_size"] # if None, do auto-detection

dataset_root       = os.path.normpath(config["dataset"]["root"])
dataset_name       = config["dataset"]["name"]
dataset_gen_method = config["dataset"]["gen_method"]
dataset_stdev      = config["dataset"]["stdev"]
dataset_param_name = config["dataset"]["param_name"]

Load `dataset_config.yaml`

In [3]:
dataset_dir = os.path.join(dataset_root, dataset_name, dataset_gen_method, dataset_stdev, dataset_param_name)
dataset_config_path = os.path.join(dataset_dir, "dataset_config.yaml")

with open(dataset_config_path, mode="r") as f_reader:
    dataset_config = yaml.load(f_reader, Loader=yaml.SafeLoader)
    
intensity = dataset_config["gen_param"]["intensity"]
drop_ratio = dataset_config["gen_param"]["drop_ratio"]

Generate `path_vars`

In [4]:
test_selected_dir = os.path.join(dataset_dir, "test", "selected")
test_drop_dir = os.path.join(dataset_dir, "test", "drop")

train_selected_dir = os.path.join(dataset_dir, "train", "selected")
train_drop_dir = os.path.join(dataset_dir, "train", "drop")

# crop_gallery_dir
test_crop_gallery_dir = os.path.join(dataset_dir, "!--- Crop Gallery/test")
train_crop_gallery_dir = os.path.join(dataset_dir, "!--- Crop Gallery/train")

# crop_gallery_class_dir
logs_path = os.path.join(dataset_dir, r"{Logs}_train_selected_summary.log")
with open(logs_path, 'r') as f_reader: class_counts: Dict[str, int] = json.load(f_reader)
for key, _ in class_counts.items():
    create_new_dir(os.path.join(test_crop_gallery_dir, key), display_in_CLI=False)
    create_new_dir(os.path.join(train_crop_gallery_dir, key), display_in_CLI=False)

Run

In [5]:
# Scan 'dark_ratio.log'
train_dark_ratio_path_list = glob(os.path.normpath(f"{dataset_dir}/*train*_dark_ratio.log"))
test_dark_ratio_path_list = glob(os.path.normpath(f"{dataset_dir}/*test*_dark_ratio.log"))


# train_dark_ratio
train_dark_ratio_dict = {}
for train_dark_ratio_path in train_dark_ratio_path_list:
    with open(train_dark_ratio_path, 'r') as f_reader: 
        train_dark_ratio = json.load(f_reader)
    train_dark_ratio_dict.update(train_dark_ratio)
train_key_list = [ key for key, _ in train_dark_ratio_dict.items() ]


# test_dark_ratio
test_dark_ratio_dict = {}
for test_dark_ratio_path in test_dark_ratio_path_list:
    with open(test_dark_ratio_path, 'r') as f_reader: 
        test_dark_ratio = json.load(f_reader)
    test_dark_ratio_dict.update(test_dark_ratio)
test_key_list = [ key for key, _ in test_dark_ratio_dict.items() ]


assert train_key_list == test_key_list

In [6]:
def make_crop_gallery(fish_name_for_dataset_list:List[str], selected_dir:str, drop_dir:str, 
                      dark_ratio_dict:Dict[str, Dict[str, float]], crop_gallery_dir:str):
    
    pbar_n_fish = tqdm(total=len(fish_name_for_dataset_list), desc="Crop Gallery ")
    fish_name_for_dataset_list.sort()
    
    for fish_name_for_dataset in fish_name_for_dataset_list:
        
        pbar_n_fish.desc = f"Generate ' {fish_name_for_dataset} ' "
        pbar_n_fish.refresh()
        
        fish_name_for_dataset_split_list = re.split(" |_|-", fish_name_for_dataset)
        
        selected_path_list = glob(os.path.normpath((f"{selected_dir}/{fish_name_for_dataset_split_list[0]}/"
                                                    f"{fish_name_for_dataset}_selected_*.tiff")))
        
        drop_path_list = glob(os.path.normpath((f"{drop_dir}/{fish_name_for_dataset_split_list[0]}/"
                                                f"{fish_name_for_dataset}_drop_*.tiff")))
        
        # read images as Dict[path, cv2.Mat]
        selected_img_dict = { img_path: cv2.imread(img_path) for img_path in selected_path_list }
        drop_img_dict = { img_path: cv2.imread(img_path) for img_path in drop_path_list }
        
        
        # draw on 'drop' images
        for path, bgr_img in drop_img_dict.items():
            
            drop_image_name = path.split(os.sep)[-1].split(".")[0]
            dark_ratio = dark_ratio_dict[fish_name_for_dataset][drop_image_name]
            
            rgb_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
            rgb_img = np.uint8(rgb_img*0.5) # suppress brightness
            
            rgb_img = Image.fromarray(rgb_img)
            draw_x_on_image(rgb_img, line_color, line_width)
            draw_drop_info_on_image(rgb_img, intensity, dark_ratio, drop_ratio,
                                    text_font_style, text_font_size,
                                    text_selected_color,
                                    text_drop_color,
                                    text_shadow_color)
            drop_img_dict[path] = cv2.cvtColor(np.array(rgb_img), cv2.COLOR_RGB2BGR)
            
        
        # draw on 'selected' images
        for path, bgr_img in selected_img_dict.items():
            
            selected_image_name = path.split(os.sep)[-1].split(".")[0]
            dark_ratio = dark_ratio_dict[fish_name_for_dataset][selected_image_name]
            
            rgb_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
            
            rgb_img = Image.fromarray(rgb_img)
            draw_drop_info_on_image(rgb_img, intensity, dark_ratio, drop_ratio,
                                    text_font_style, text_font_size,
                                    text_selected_color,
                                    text_drop_color,
                                    text_shadow_color)
            selected_img_dict[path] = cv2.cvtColor(np.array(rgb_img), cv2.COLOR_RGB2BGR)
            
        
        both_img_dict = deepcopy(selected_img_dict)
        both_img_dict.update(drop_img_dict)
        sorted_both_img_dict = OrderedDict(sorted(list(both_img_dict.items()), key=lambda x: sortFishNameForDataset(x[0])))
        both_img_list = [ img for _, img in sorted_both_img_dict.items() ]
        
        # plot with 'Auto Row Calculation'
        kwargs_plot_with_imglist_auto_row = {
            "img_list"   : both_img_list,
            "column"     : column,
            "fig_dpi"    : 200,
            "figtitle"   : (f"( Crop Condition ) {fish_name_for_dataset} : {both_img_list[-1].shape[:2]}, "
                            f"drop_ratio = {dataset_config['gen_param']['drop_ratio']}, drop : {len(drop_path_list)}") ,
            "save_path"  : f"{crop_gallery_dir}/{fish_name_for_dataset_split_list[0]}/{fish_name_for_dataset}_crop.png",
            "show_fig"   : False
        }
        plt_show.plot_with_imglist_auto_row(**kwargs_plot_with_imglist_auto_row)
        
        pbar_n_fish.update(1)
        pbar_n_fish.refresh()
    
    pbar_n_fish.close()

In [ ]:
make_crop_gallery(train_key_list, train_selected_dir, train_drop_dir, train_dark_ratio_dict, train_crop_gallery_dir)
make_crop_gallery(test_key_list, test_selected_dir, test_drop_dir, test_dark_ratio_dict, test_crop_gallery_dir)